<img src=https://static.wixstatic.com/media/f6bd9a_6a972d9b31324653bf198c38ce94339e~mv2.png>

# Carrusel de RDBMS
## Ejercicios de SQL resueltos

En este cuaderno resuelvo los ejercicios del 11 al 20 del documento: 'Desarrollo de Aplicaciones en Entornos de Cuarta Generación y Herramientas CASE: Ejercicios de SQL.'

Para ello utilizo una version de la base de datos -que nombre "practica_hospital"- modificada para ser compatible en esta ocasion, para **SQLite**, siguiendo la estructura del libro.


Tanto el diagrama de **Entidad-Relacion**, como el **Script** para cargar la base de datos (que permite, se pueda replicar el ejercicio) se encuentran disponibles en el mismo repositorio. Por la forma en que SQLite administra las bases de datos, primero se crea la instancia y luego se corre el script

#### Celda para configurar el entorno

In [ ]:
# !pip install pandas

#### Se importan las librerias y se crea la conexion. La base de datos debe estar en el directorio.

In [1]:
# importar las librerias
import sqlite3
import pandas as pd

# establecer la conexion
conn = sqlite3.connect('practica_hospital.db')

#### Realizamos una prueba y se Inspecciona la base de datos "practica_hospital"

In [2]:
# Definir consulta en SQL compatible con SQLite
try:
    QUERY = """SELECT name FROM sqlite_master WHERE type='table';"""
# Ejecutar la consulta y obtener los resultados en un DataFrame
    df = pd.read_sql_query(QUERY, conn)
# Mostrar los primeros resultados
    print("La conexión se ha realizado con éxito.\n")
    
    print(df)
except Exception as e:
    print("Hubo un error al realizar la conexión:", e)

La conexión se ha realizado con éxito.

              name
0         HOSPITAL
1             SALA
2        PLANTILLA
3        OCUPACION
4           DOCTOR
5          ENFERMO
6  sqlite_sequence
7         EMPLEADO
8     DEPARTAMENTO


#### Inspeccion a las tablas de la base de datos "practica_hospital"
#### Con la finalidad de observar su estructura y tipo de datos

In [3]:
# se excluye el nombre de la tabla del sistema
indice = df[df['name'] == 'sqlite_sequence'].index
df = df.drop(indice)
for valor in df['name']:
    QUERY = f'PRAGMA TABLE_INFO({valor});'
    df1 = pd.read_sql_query(QUERY, conn)
    print(f'El contenido de la tabla {valor}\n')
    print(df1)
    print()

El contenido de la tabla HOSPITAL

   cid          name     type  notnull dflt_value  pk
0    0  HOSPITAL_COD  INTEGER        1       None   1
1    1        NOMBRE     TEXT        1       None   0
2    2     DIRECCION     TEXT        1       None   0
3    3      TELEFONO     TEXT        1       None   0
4    4      NUM_CAMA  INTEGER        1       None   0

El contenido de la tabla SALA

   cid          name     type  notnull dflt_value  pk
0    0       SALA_ID  INTEGER        0       None   1
1    1  HOSPITAL_COD  INTEGER        1       None   0
2    2      SALA_COD  INTEGER        1       None   0
3    3        NOMBRE     TEXT        0       None   0
4    4      NUM_CAMA  INTEGER        0       None   0

El contenido de la tabla PLANTILLA

   cid          name     type  notnull dflt_value  pk
0    0  HOSPITAL_COD  INTEGER        1       None   0
1    1      SALA_COD  INTEGER        1       None   0
2    2   EMPLEADO_NO  INTEGER        0       None   1
3    3      APELLIDO     TEXT   

### Resolucion de los ejercicios numerados del 11 al 20

##### OPERADORES Y FUNCIONES DE FECHAS

##### 11. Encuentre a todas las enfermeras y enfermeros con indicación del salario mensual de cada uno

In [4]:
QUERY = """
SELECT APELLIDO, FUNCION, ROUND(SALARIO/12) AS SALARIO_MENSUAL
FROM PLANTILLA
WHERE UPPER(FUNCION) IN ("ENFERMERO", "ENFERMERA");
"""
df = pd.read_sql_query(QUERY, conn)
df

,APELLIDO,FUNCION,SALARIO_MENSUAL
0,Higueras D.,Enfermera,167083.0
1,Hernandez J.,Enfermero,229583.0
2,Diaz B.,Enfermera,188500.0
3,Rivera G.,Enfermera,135500.0
4,Carlos R.,Enfermera,176583.0
5,Bocina G.,Enfermero,136500.0
6,Frank H.,Enfermera,210166.0


##### 12. Que fecha fue hace tres semanas?

In [5]:
QUERY = """
SELECT DATE('now','-21 day') AS FECHA_HACE_3;
"""
df = pd.read_sql_query(QUERY, conn)
df

,FECHA_HACE_3
0,2024-02-26


##### 13.Se pide el nombre, oficio y fecha de alta del personal del departamento 20 que ganan mas de 150000 ptas. mensuales.

In [6]:
QUERY = """
SELECT APELLIDO, OFICIO, STRFTIME('DIA %d del mes %m del año %Y', FECHA_ALTA) AS FECHA
FROM EMPLEADO
WHERE SALARIO > 150000 AND DEPT_NO = 20;
"""
df = pd.read_sql_query(QUERY, conn)
df

,APELLIDO,OFICIO,FECHA
0,Jimenez,Director,DIA 02 del mes 04 del año 1981
1,Gil,Analista,DIA 09 del mes 12 del año 1982
2,Fernandez,Analista,DIA 03 del mes 12 del año 1981


##### 14. Se pide el nombre, oficio y el día de la semana en que han sido dados de alta los empleados de la empresa, 
##### pero solo de aquellos cuyo día de alta haya sido entre martes y jueves. Ordenado por oficio.

In [7]:
QUERY = """
SELECT APELLIDO, OFICIO, CASE 
	WHEN STRFTIME('%w', FECHA_ALTA) = '3' THEN 'MARTES'
	WHEN STRFTIME('%w', FECHA_ALTA) = '4' THEN 'MIERCOLES'
	WHEN STRFTIME('%w', FECHA_ALTA) = '5' THEN 'JUEVES'
	ELSE '.'
END AS FECHA
FROM EMPLEADO
WHERE STRFTIME('%w', FECHA_ALTA) = '3' OR STRFTIME('%w', FECHA_ALTA) = '4' OR STRFTIME('%w', FECHA_ALTA) = '5'
ORDER BY OFICIO;
"""
df = pd.read_sql_query(QUERY, conn)
df

,APELLIDO,OFICIO,FECHA
0,Gil,Analista,MIERCOLES
1,Fernandez,Analista,MIERCOLES
2,Jimenez,Director,MIERCOLES
3,Negro,Director,JUEVES
4,Sanchez,Empleado,MARTES
5,Alonso,Empleado,MARTES
6,Jimeno,Empleado,MIERCOLES
7,Arroyo,Vendedor,JUEVES


##### 15. Para todos los empleados, el día en que fueron dados de alta en la empresa debe estar ordenada por el día 
##### de la semana (Lunes, Martes ... Viernes) . Los días no laborables serán "Fin de semana".*/

In [8]:
QUERY = """
SELECT APELLIDO, OFICIO, CASE 
	WHEN STRFTIME('%w', FECHA_ALTA) = '2' THEN 'LUNES'
	WHEN STRFTIME('%w', FECHA_ALTA) = '3' THEN 'MARTES'
	WHEN STRFTIME('%w', FECHA_ALTA) = '4' THEN 'MIERCOLES'
	WHEN STRFTIME('%w', FECHA_ALTA) = '5' THEN 'JUEVES'
	WHEN STRFTIME('%w', FECHA_ALTA) = '6' THEN 'VIERNES'
	ELSE 'FIN DE SEMANA'
END AS FECHA
FROM EMPLEADO
ORDER BY STRFTIME('%w', FECHA_ALTA);
"""
df = pd.read_sql_query(QUERY, conn)
df

,APELLIDO,OFICIO,FECHA
0,Sala,Vendedor,FIN DE SEMANA
1,Arenas,Vendedor,FIN DE SEMANA
2,Cerezo,Director,LUNES
3,Rey,Presidente,LUNES
4,Tovar,Vendedor,LUNES
5,Sanchez,Empleado,MARTES
6,Alonso,Empleado,MARTES
7,Jimenez,Director,MIERCOLES
8,Gil,Analista,MIERCOLES
9,Jimeno,Empleado,MIERCOLES


#### CLÁUSULA GROUP BY 
##### 16.Encontrar el salario medio de los Analistas.

In [9]:
QUERY = """
SELECT OFICIO, AVG (SALARIO) AS SALARIO_MEDIO
FROM EMPLEADO
GROUP BY OFICIO
HAVING OFICIO = 'Analista'
"""
df = pd.read_sql_query(QUERY, conn)
df

,OFICIO,SALARIO_MEDIO
0,Analista,390000.0


##### 17. Encontrar el salario mas alto y el salario mas bajo de la tabla de empleados, así como la diferencia entre ambos.

In [10]:
QUERY = """
SELECT MAX(SALARIO) AS SALARIO_MAX, MIN(SALARIO) AS SALARIO_MIN, MAX(SALARIO)-MIN(SALARIO) AS SALARIO_DIFERENCIA
FROM EMPLEADO
"""
df = pd.read_sql_query(QUERY, conn)
df

,SALARIO_MAX,SALARIO_MIN,SALARIO_DIFERENCIA
0,650000,104000,546000


##### 18. Calcular el numero de oficios diferentes que hay, en total, en los departamentos 10 y 20 de la empresa.

In [11]:
QUERY = """
SELECT COUNT(DISTINCT(OFICIO)) AS TOTAL_OFICIOS
FROM EMPLEADO
WHERE DEPT_NO IN (10,20);
"""
df = pd.read_sql_query(QUERY, conn)
df

,TOTAL_OFICIOS
0,4


##### 19. Calcular el numero de personas que realizan cada oficio en cada departamento.

In [12]:
QUERY = """
SELECT DEPT_NO, OFICIO, COUNT (OFICIO) AS NUM_PERSONAS
FROM EMPLEADO
GROUP BY DEPT_NO, OFICIO;
"""
df = pd.read_sql_query(QUERY, conn)
df

,DEPT_NO,OFICIO,NUM_PERSONAS
0,10,Director,1
1,10,Empleado,1
2,10,Presidente,1
3,20,Analista,2
4,20,Director,1
5,20,Empleado,2
6,30,Director,1
7,30,Empleado,1
8,30,Vendedor,4


##### 20. Buscar que departamentos tienen mas de cuatro personas trabajando.

In [13]:
QUERY = """
SELECT DEPT_NO, COUNT(OFICIO) AS PERSONAS_TRABAJANDO
FROM EMPLEADO
GROUP BY DEPT_NO
HAVING COUNT(OFICIO) > 4;
"""
df = pd.read_sql_query(QUERY, conn)
df

,DEPT_NO,PERSONAS_TRABAJANDO
0,20,5
1,30,6


In [14]:
# cerrar la conexion
conn.close()

# Autor
### Oscar Gutierrez Leal
15 - 03 - 2024